<a href="https://colab.research.google.com/github/Nzaya/Mchezopesa-Ltd/blob/master/Moringa_Data_Science_Core_W6_Independent_Project_2019_06_Sharon_Nzaya_Python_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

  # QUESTION
  
  
  You have been recruited as a football analyst in a company - Mchezopesa Ltd and tasked to accomplish the task below.

A Prediction result of a game between team 1 and team 2, based on who's home and who's away, and on whether or not the game is friendly (include rank in your training).

# METRICS OF SUCCESS

# CONTEXT

# EXPERIMENTAL DESIGN TAKEN

# Importing necessary libraries

In [0]:
import pandas as pd
import numpy as np

# Loading our dataset

In [0]:
#Loading our ranking dataset

ranking=pd.read_csv('/content/fifa_ranking.csv')
ranking.head()

In [0]:
#Loading our results dataset

results=pd.read_csv('/content/results.csv')
results.head()